In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model="gpt-4.1-2025-04-14",
    temperature=0.3,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [2]:
examples = [
    {
        "question": "I want a summary of the movie 'Top Gun' only written by emojis.",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "I want a summary of the movie 'The Godfather' only written by emojis.",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "I want a summary of the movie 'Interstellar' only written by emojis.",
        "answer": "🧑‍🚀🚀⭐",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie lover. Basically, when given question about a movie, you reply with exactly 3 emoji that represents the movie well. But when asked the movie that you answered before, you must reply with the same 3 emoji that you answered before. Also, when human asks you about movies that you remember, you reply every movie title that you remember.",
        ),
        MessagesPlaceholder(variable_name="history"),
        chat_prompt,
        (
            "human",
            "{question}",
        ),
    ]
)

In [3]:
memory = ConversationBufferMemory(
    return_messages=True,
    input_key="question",
    output_key="answer",
)


def save(input, output):
    memory.save_context(
        {"question": input},
        {"answer": output},
    )


save(examples[0]["question"], examples[0]["answer"])
save(examples[1]["question"], examples[1]["answer"])
save(examples[2]["question"], examples[2]["answer"])


def load_memory(x):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

C:\Users\trace\AppData\Local\Temp\ipykernel_34360\2654005382.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [4]:
def invoke_by_movie(movie):
    question = f"I want a summary of the movie '{movie}' only written by emojis."
    result = chain.invoke({"question": question})
    save(question, result.content)


def invoke_by_question(question):
    result = chain.invoke({"question": question})
    save(question, result.content)

In [5]:
invoke_by_movie("Avengers: Endgame")

🦸‍♂️🧤💥

In [6]:
invoke_by_movie("Dune")

🏜️🪱👑

In [7]:
invoke_by_question("Do you remember the movies I asked before?")

Top Gun  
The Godfather  
Interstellar  
Avengers: Endgame  
Dune

In [8]:
invoke_by_movie("Interstellar")

🧑‍🚀🚀⭐

In [9]:
invoke_by_movie("Top Gun")

🛩️👨‍✈️🔥

In [10]:
invoke_by_movie("The Godfather")

👨‍👨‍👦🔫🍝

In [11]:
invoke_by_movie("The Amazing Spider-man")

🕷️🧑‍🔬🏙️

In [12]:
invoke_by_question("Do you remember the movies I asked before?")

Top Gun  
The Godfather  
Interstellar  
Avengers: Endgame  
Dune  
The Amazing Spider-man